# This notebook builds bash script for transcoding VRTs to COGs using marblecutter-tools


In [ ]:
## Collect imagery Summary

from os import walk
import os
f = []
ftype = []
mypath = '/raid/nfs/data/Datasets/CosmiQ_General_Study/CORE3D-Public-Data/Satellite-Images/'
for (dirpath, dirnames, filenames) in walk(mypath):
    print(os.path.basename(dirpath))
    
    f.extend([os.path.join(dirpath, file) for file in filenames])
    ftype.extend([os.path.basename(dirpath) for file in filenames])

basename_list = []
ext_list      = []

cell_list = {}
idstr_list = []
imtype_list = []
sloc_list = []
fNew = []
for file in f:
    try:
        basename = os.path.basename(file)
        basesplit = basename.split('-')
        idStr = basesplit[0]
        imType = basesplit[1]


        idstr_list.append(basesplit[0])
        imtype_list.append(basesplit[1])
        basename_list.append(os.path.basename(file))
        ext_list.append(os.path.splitext(file)[1])
        sloc_list.append(file.replace('/raid/nfs/data/Datasets/CosmiQ_General_Study/CORE3D-Public-Data/', 's3://spacenet-dataset/CORE3D-Public-Data/')
                        )
        fNew.append(file)
    except:
        print("error")
    
import pandas as pd

dataList = {'idstr': idstr_list,
           "imtype": imtype_list,
           "basename": basename_list,
           "ext": ext_list,
           "filepath": fNew,
            "s3loc": sloc_list
           }
df = pd.DataFrame(dataList) 

In [ ]:
dockerCommandList = []
dockerCommandString = "time docker run -v /raid/nfs/data:/raid/nfs/data quay.io/mojodna/marblecutter-tools transcode.sh {} {}"
for vrt_path in (df[df['ext']=='.vrt']['filepath'].values):
    cog_path = vrt_path.replace('.vrt', '_lv1.tif')
    dockerCommandList.append(dockerCommandString.format(vrt_path, cog_path))

with open('/raid/nfs/data/Datasets/CosmiQ_General_Study/CORE3D-Public-Data/create_cogs.sh', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(dockerCommandList))

In [ ]:
df['ext'].unique()